## **Install transformers dataset**
```
!pip install transformers datasets accelerate ray[tune] optuna -U
```
This line of code is used to install several Python libraries needed for working with NLP models and optimizing them. The exclamation mark at the beginning allows the command to run directly in a notebook. transformers is a library that provides pre-trained models like BERT and RoBERTa for tasks such as text classification and sentiment analysis, while datasets helps to easily access and manage NLP datasets. accelerate makes training faster and more efficient by allowing models to run on multiple devices or use mixed-precision. ray[tune] and optuna are both tools for hyperparameter optimization, which means they help find the best settings for a model to perform well. Finally, the -U flag ensures that all these packages are updated to their latest versions.

In [ ]:
!pip install transformers datasets accelerate ray[tune] optuna -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently tak

## **Setup and Installation**

```
import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")
```
This part of the code sets up the environment for training a text classification model. First, it imports the necessary libraries: torch for deep learning, numpy and pandas for data handling, datasets for NLP datasets, transformers for working with pre-trained models, sklearn for data splitting and evaluation metrics, and optuna for hyperparameter optimization. The set_seed(42) function is used to make the results reproducible, meaning the same results can be obtained every time the code runs. The code also checks whether a GPU is available for training; if so, it uses the GPU to speed up computation, otherwise it defaults to the CPU, which is slower but still works.

-----
## **Data Preparation**
```
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']
df = df.rename(columns={"label": "labels"})

train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=10000, random_state=42)
eval_df = eval_df.sample(n=2000, random_state=42)

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
```
This code loads and prepares the dataset for training a text classification model. First, it tries to read a CSV file named Mental-Health-Twitter.csv and stops execution with an error message if the file is not found. Missing or empty text entries in the post_text column are removed, and the target column is renamed to labels for consistency. The dataset is then split into training (90%) and evaluation (10%) sets, and a subset of 10,000 training samples and 2,000 evaluation samples is randomly selected to reduce computation time. The distribution of labels in both sets is displayed to ensure balance. Next, the training and evaluation data are converted into Hugging Face Dataset objects. A pre-trained RoBERTa model (margotwagner/roberta-psychotherapy-eval) is loaded along with its tokenizer. The tokenize_function converts each text into token IDs and attention masks, truncating or padding sequences to a maximum length of 128 tokens.The tokenized datasets are formatted for PyTorch, keeping only the necessary columns: input_ids, attention_mask, and labels, ready for model training.

-----
## **Model and Metrics**
```
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# --- 3b. HYPERPARAMETER SPACE INCLUDING GRADIENT ACCUMULATION ---
def tune_hp(trial):
    learning_rate = trial.suggest_categorical("learning_rate", [2e-5, 3e-5])
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    weight_decay = trial.suggest_categorical("weight_decay", [0.01, 0.03])
    num_train_epochs = trial.suggest_categorical("num_train_epochs", [2, 3])
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])
    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
        "gradient_accumulation_steps": gradient_accumulation_steps
    }
  ```
This code defines the model, evaluation metrics, and the hyperparameter search space for training. The model_init() function loads the pre-trained RoBERTa model for sequence classification with two output labels (depression or not) and moves it to the selected device (GPU or CPU). The compute_metrics() function calculates key evaluation metrics: accuracy, F1-score, precision, and recall by comparing the model’s predictions with the true labels. The tune_hp() function defines the hyperparameter search space for optimization. It specifies possible values for learning rate, batch size per device, weight decay, number of training epochs, and gradient accumulation steps. These hyperparameters are then tested during training to find the combination that yields the best model performance.

-----
## **Training Arguments**
```
training_args = TrainingArguments(
    output_dir="./grid_search_results_mental_health",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none",
    num_train_epochs=3,
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=200,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    adam_epsilon=1e-7,
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

def optuna_hp_objective(metrics):
    return metrics["eval_f1"]
```
This code sets up the training configuration and trainer for the model. The TrainingArguments specify how the model will be trained and evaluated. The results and logs are saved in designated folders, and the model is evaluated and saved at the end of each epoch. Only the best model based on F1-score is kept, and mixed-precision training (fp16) is enabled if a GPU is available to speed up computation. Other settings include the number of training epochs, warmup steps for the optimizer, logging frequency, number of CPU workers for loading data, and the Adam optimizer’s epsilon value. The Trainer class from Hugging Face is then initialized with the model, training arguments, tokenized datasets, evaluation metrics, and tokenizer. Finally, the optuna_hp_objective() function is defined to return the evaluation F1-score, which is used to guide the hyperparameter optimization process.

-----
## **Execution of Grid Search**
```
print("\n--- Starting Hyperparameter Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

NUM_TRIALS = 20
print(f"Number of trials to run: {NUM_TRIALS}")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=tune_hp,
    direction="maximize",
    n_trials=NUM_TRIALS,
    compute_objective=optuna_hp_objective,
)

print("\n--- Hyperparameter Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        gradient_accumulation_steps=best_hps["gradient_accumulation_steps"],   
        warmup_steps=100,
        logging_dir="./final_logs",
        logging_steps=200,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
        adam_epsilon=1e-7,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")
  ```
This code performs hyperparameter optimization using Optuna and trains the final model with the best parameters found. First, it announces the start of the search and specifies that the F1-score will be used to determine the best model. A total of 20 trials are run, where each trial tests a different combination of hyperparameters defined in the tune_hp() function. The trainer.hyperparameter_search() method performs the optimization, and the best hyperparameters and evaluation metrics are printed. Once the best hyperparameters are identified, the code sets up new TrainingArguments using these optimal values, including learning rate, batch size, number of epochs, weight decay, and gradient accumulation steps. A new Trainer is initialized with these settings, and the final model is trained on the tokenized training dataset. After training, the model is evaluated on the evaluation dataset, and its performance metrics are displayed. The trained model is saved in the folder ./final_model_mental_health for future use.

In [ ]:
# 1. SETUP AND INSTALLATION
# Run this command first in your Colab notebook:
# !pip install transformers datasets accelerate ray[tune] optuna pandas -U

import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")

# --- 2. DATA PREPARATION ---
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']
df = df.rename(columns={"label": "labels"})

train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=10000, random_state=42)
eval_df = eval_df.sample(n=2000, random_state=42)

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])

# --- 3. MODEL AND METRICS ---
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# --- 3b. HYPERPARAMETER SPACE INCLUDING GRADIENT ACCUMULATION ---
def tune_hp(trial):
    learning_rate = trial.suggest_categorical("learning_rate", [2e-5, 3e-5])
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    weight_decay = trial.suggest_categorical("weight_decay", [0.01, 0.03])
    num_train_epochs = trial.suggest_categorical("num_train_epochs", [2, 3])
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])
    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
        "gradient_accumulation_steps": gradient_accumulation_steps
    }

# --- 4. TRAINING ARGUMENTS ---
training_args = TrainingArguments(
    output_dir="./grid_search_results_mental_health",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none",
    num_train_epochs=3,
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=200,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    adam_epsilon=1e-7,
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

def optuna_hp_objective(metrics):
    return metrics["eval_f1"]

# --- 5. EXECUTION OF Grid SEARCH ---
print("\n--- Starting Hyperparameter Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

NUM_TRIALS = 20
print(f"Number of trials to run: {NUM_TRIALS}")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=tune_hp,
    direction="maximize",
    n_trials=NUM_TRIALS,
    compute_objective=optuna_hp_objective,
)

print("\n--- Hyperparameter Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        gradient_accumulation_steps=best_hps["gradient_accumulation_steps"],
        warmup_steps=100,
        logging_dir="./final_logs",
        logging_steps=200,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
        adam_epsilon=1e-7,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")


Using GPU: Tesla T4
Dataset loaded successfully.
Using 10000 training samples and 2000 evaluation samples.
Train label distribution:
labels
1    0.5004
0    0.4996
Name: proportion, dtype: float64
Eval label distribution:
labels
1    0.5
0    0.5
Name: proportion, dtype: float64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

[I 2025-11-08 10:51:56,270] Trial 18 finished with value: 0.9108121574489287 and parameters: {'learning_rate': 3e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.01, 'num_train_epochs': 3, 'gradient_accumulation_steps': 1}. Best is trial 7 with value: 0.9108121574489287.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.317100,0.261968,0.883000,0.888889,0.846293,0.936000
2,0.206000,0.323081,0.893500,0.899195,0.853549,0.950000
3,0.139800,0.345660,0.910500,0.910812,0.907646,0.914000


[I 2025-11-08 11:00:04,283] Trial 19 finished with value: 0.9108121574489287 and parameters: {'learning_rate': 3e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.01, 'num_train_epochs': 3, 'gradient_accumulation_steps': 1}. Best is trial 7 with value: 0.9108121574489287.



--- Hyperparameter Search Complete ---

BEST HYPERPARAMETERS FOUND:
BestRun(run_id='7', objective=0.9108121574489287, hyperparameters={'learning_rate': 3e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.01, 'num_train_epochs': 3, 'gradient_accumulation_steps': 1}, run_summary=None)

Best Hyperparameters:
  learning_rate: 3e-05
  per_device_train_batch_size: 16
  weight_decay: 0.01
  num_train_epochs: 3
  gradient_accumulation_steps: 1


AttributeError: 'BestRun' object has no attribute 'metrics'